# Introducción

Esta propuesta se basa en **clustering** dado que se busca encontrar grupos con similitudes por sus gustos, para lo cual es vital el rating para cada anime por cada usuario. El objetivo final es que los usuarios queden agrupados según como valoraron los distintos animes.

Para este problema se propone el algoritmo de clústering **K-Means** dada su extensa aplicación y su rápida ejecución. Además se propone antes de utilizar K-Means, reducir la dimensionalidad de los datos con el fin de disminuir los tiempos de entrenamiento.

# Estrategia

En esta problemática de **aprendizaje no supervisado**, se buscar juntar ambas tablas mediante el *anime_id* ejecutando una operación de **INNER-JOIN**. Luego a partir de esta tabla se busca generar una cross-table obteniendo una tabla de tantas filas como usuarios existan, y tantas columnas como animes se encuentren en los datos, teniendo como dato en cada coordenada *i,j* la valoración del usuario *i* para el anime *j*.

# Sistema de recomendación de anime

Ambas tablas contienen información de preferencias de 73.516 usuario en 12.294 anime (series de dibujos animados orientales). Cada usuario puede agregar un anime a su lista y darle un rating (de 0 a 10). La descripción de la data es la siguiente:

Anime.csv 
- anime_id : id único del anime (de la página myanimelist.net) 
- name : nombre del anime 
- genre : lista de generos separados por coma del anime 
- type : TV, movie (de película), OVA, etc… 
- episodes : cantidad de episodios del show (1 si es película) 
- rating : rating promedio (de 1-10) para este anime 
- members : numero de miembros de la comunidad que están en el grupo del anime

Rating.csv 
- user_id : id del usuario generado aleatoriamente 
- anime_id : el anime que el usuario rankeo 
- rating : el rating entre 1 y 10 que el usuario asignó al anime ( -1 si el usuario vio el anime pero no le asignó puntaje)

El objetivo del desafío es desarrollar un sistema de recomendación que permita sugerir anime que los usuarios no han visto, en base a sus ratings previos. Es muy importante justificar la elección del sistema (o modelo), el trabajo previo de la data (EDA) y la documentación de lo que se hizo (no es necesario un informe, pero si comentar porqué se tomaron las decisiones que se tomaron; por ej eliminar una variable o eliminar registros missing, etc).

In [351]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

pd.options.display.max_rows = 10

## Carga de datos

In [ ]:
path_data_folder = '../input'
name_anime_data = r'anime.csv'
name_rating_data = r'rating.csv'

In [ ]:
df_anime = pd.read_csv(path_data_folder+'/'+name_anime_data,index_col='anime_id', sep=',')
df_rating = pd.read_csv(path_data_folder+'/'+name_rating_data, sep=',')

## Características datos del dataframe *df_anime*

In [ ]:
df_anime.shape

In [ ]:
df_anime.dtypes

Los episodios también deben ser tratados como números enteros. Veamos los posibles valores de esta columna

## Tratamiento de datos para el dataframe *df_anime*

### Tratamiento columna episodes

In [ ]:
df_anime.episodes.value_counts()

Como podemos ver, existen valores 'Unknown', para los cuales daremos un valor entero negativo constante. En este caso **-1**. A pesar de ser una minima parte que conforma este grupo con respecto al total, se prefiere no borrar, ni tampoco utilizar media, mediana o moda, puesto que existen valores atípicos, por lo que se genera un nuevo grupo distintivo.

In [ ]:
df_anime.loc[df_anime.episodes == 'Unknown', 'episodes'] = -1

In [ ]:
df_anime.episodes = pd.to_numeric(df_anime.episodes)

In [ ]:
df_anime.dtypes

Como podemos ver, sin ningún problema la columna fue *casteada* a valor entero.
- NOTA: Casting es convertir el tipo de dato que posee una variable.

In [ ]:
df_anime.describe()

In [ ]:
df_anime.episodes.describe()

In [ ]:
df_anime.episodes.hist()

### Tratamiento columna genre

Inicialmente borraria los anime sin género, ya que considero que es demasiado importante para la recomendación
 - Sin género no existe relación directa con los gustos del usuario
 - Sin embargo podría darse una relación implícita en el rating de otros usuarios con el fin de categorizar a dichos anime. Así mismo de acuerdo al gusto de los usuarios podrían caer en alguna categoría y obtener dicha etiqueta.

In [ ]:
df_anime.loc[df_anime.genre.isnull()]

Si bien la cantidad de registros en la tabla anime con valores null es mínima, de igual forma estos tienen un rating asignado y es importante considerarlos por lo que se toma la decisión de crear un nuevo valor **'No Genre'** para estos valores.

In [ ]:
df_anime.genre.fillna('No Genre',inplace=True)

In [ ]:
df_anime.count()

**IMPORTANTE**: Un **mínimo** de registros tiene valor NaN en *type* y también en *episodes* y *rating*.

### Tratamiento columna *type*

In [ ]:
df_anime.loc[df_anime.type.isna()]

Como podemos ver, en los registros(o rows) con valor NaN en la columna *type* coincide que en todas existe falta de información en las columnas *episodes* y *rating*, siendo esta última razón para ser eliminada siguiendo la estrategia. Sin embargo compararemos el número de miembros de cada uno de estos registros con respecto a los demás.

In [ ]:
df_anime.members.hist()

In [ ]:
pd.Series(df_anime.members.describe(),dtype='int64')

Borramos los valores NaN

In [ ]:
df_anime = df_anime.dropna(0, subset=['type'])

In [ ]:
pd.isnull(df_anime).sum()

In [ ]:
df_anime.describe()

In [ ]:
df_anime.count()

### Tratamiento columna rating

In [ ]:
df_anime.loc[pd.isna(df_anime.rating)]

Como podemos ver, lo más probable es que estos anime aún no sean vistos, por lo que podrían no ser recomendamos simplemente (dado que no tienen rating) o ser considerados por su género, puesto que claramente los géneros pertenecientes a un anime son una relación con otros. Sin embargo estos valores NaN en la columna rating sean tratados o no, serán eliminados al ejecutar la operación **INNER JOIN** con la tabla(o dataframe) rating, puesto que no existen usuarios en la tabla rating que hayan evaluado al anime de la tabla anime.
Para esta ocasión excepcional, tomo la decisión de no considerar a los valores NaN en rating, dado la mínima parte que conforman con respecto al total. Sin embargo, si fuesen una gran parte(o mayor), utilizaría la estrategia de ejecutar una operación **OUTER JOIN** entre ambas tablas y trataría de otra manera los valores NaN en rating.

In [ ]:
df_anime.count()

In [ ]:
nro_total_ratings = df_anime.rating.shape[0]
nro_total_ratings

In [ ]:
df_anime.loc[df_anime.rating.isnull()].shape

In [ ]:
nro_nulls = pd.isna(df_anime.rating).sum()
nro_nulls

## Gráfico ratings válidos vs nulls

In [ ]:
impr = ["validos", "null"]
vol = [nro_total_ratings-nro_nulls, nro_nulls]
expl =(0, 0.05)
plt.pie(vol, explode=expl, labels=impr, autopct='%1.1f%%', shadow=True)
plt.title("Impresión", bbox={"facecolor":"0.8", "pad":10})
plt.legend()

A pesar de ser una cantidad tan baja de valores null, como antes ya se explicó, con la estrategia seguida se eliminan si o sí dado que se busca generar un dataset mediante una operación **INNER JOIN** entre ambas tablas.

In [ ]:
df_anime = df_anime.dropna(0, subset=['rating']) # borramos las filas con valores de rating na

In [ ]:
pd.isna(df_anime).sum()

In [ ]:
df_anime.index.is_unique # nos aseguramos que el índice sea único

In [ ]:
df_anime.shape

In [ ]:
df_anime.count()

In [ ]:
df_anime[['episodes', 'rating']].describe()

## Interesante ver donde está concentrada la mayoría de los ratings

In [ ]:
df_anime['rating'].hist()

In [ ]:
df_anime.loc[df_anime.rating > 9]

## Tratamiento de datos del dataframe *df_rating*

### Característica de los datos del dataframe *df_rating*

In [ ]:
df_rating.shape

In [ ]:
df_rating.count()

In [ ]:
df_rating.dtypes

In [ ]:
pd.isnull(df_rating).sum()

In [ ]:
df_rating.head()

### Tratamiento datos columna *rating*

In [ ]:
np.round(df_rating[['rating']].describe(), 2)

In [ ]:
df_rating.rating.hist()

In [ ]:
df_rating[['user_id', 'rating']].groupby(['user_id']).mean().merge(
    df_rating[['user_id', 'rating']].groupby(
    ['user_id']).count(),left_on='user_id', right_on='user_id') # veamos el rating promedio y el numero de personas

In [ ]:
df_rating.loc[(df_rating.user_id == 73515)]['rating'].hist()

In [ ]:
df_rating.loc[(df_rating.user_id == 73515)&(df_rating.rating>=1)]['rating'].hist()

# Decisión para la columna *rating*

Si bien una gran parte de este dataframe se compone de valores -1(usuario vió pero no evaluó) estos estan distribuidos por usuario en pequeñas porciones. Tomando en cuenta que el objetivo es encontrar similitudes entre los gustos de los usuarios, se toma le decisión de rellenar con el promedio de sus ratings **sin** considerar los valores -1 para dicho cálculo.

Verificamos que no existan valores no válidos como menores 0(pero mayores a -1) o mayores a 10.

In [ ]:
df_rating.loc[(df_rating.rating < 0)&(df_rating.rating > -1)|(df_rating.rating > 10)]

In [ ]:
df_rating

In [ ]:
df_rating = df_rating.merge(df_rating[['user_id', 'rating']].loc[df_rating.rating >=1].groupby(['user_id']).mean(), on='user_id',
               suffixes=('', '_prom'))

In [ ]:
df_rating

In [ ]:
df_rating['rating'].loc[df_rating.rating == -1] = df_rating['rating_prom']

In [ ]:
df_rating.loc[df_rating.user_id == 73515].rating.value_counts()

In [ ]:
df_rating[['user_id', 'rating']].groupby(['user_id']).mean().hist()

In [ ]:
df_rating['rating'].hist()

# Join

Una vez tratados los datos de ambos dataframe se ejecuta la operación **INNER-JOIN** entre ambos con el objetivo de relacionar el anime con la valoración de cada usuario.

In [ ]:
df_inner = df_rating.merge(df_anime.sample(n=100),on='anime_id', how='inner')

In [ ]:
df_inner

In [ ]:
df_inner.shape

In [ ]:
pd.isna(df_inner).sum()

In [ ]:
df_inner

De ahora en adelante para reducir los tiempos para las pruebas trabajaremos con un dataframe de muestra obtenido aleatoriamente.

In [ ]:
df_inner_sample = df_inner

In [ ]:
df_inner_sample

In [ ]:
df_inner_sample.shape

Dado que el objetivo es ejecutar el algoritmo **K-Means** con la valoración que le dio cada usuario a cada anime es que hacemos una tabla cruzada de la siguiente manera.

In [ ]:
df_rating_animes = pd.crosstab(df_inner_sample['user_id'], df_inner_sample['name'], values=df_inner_sample['rating_x'], aggfunc=['mean'])

El nuevo dataframe contiene las valoraciones que dio cada usuario para cada anime.

In [ ]:
df_rating_animes

In [ ]:
df_rating_animes = df_rating_animes.fillna(value=0.0)

In [ ]:
df_rating_animes

In [ ]:
df_rating_animes.shape

Dada la gran cantidad de columnas se procede a ejecutar *PCA* con el objetivo de reducir la dimensionalidad de los datos teniendo periodos de entrenamiento más cortos.

In [ ]:
scaler = MinMaxScaler(feature_range=[0, 1])
data_rescaled = scaler.fit_transform(df_rating_animes)

In [ ]:
data_rescaled

In [ ]:
data_rescaled.shape

In [ ]:
pca = PCA().fit(data_rescaled)

# Determinemos el número de componentes para PCA

In [ ]:
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza (%)') #for each component
plt.title('Explained Variance')
plt.show()

De las 89 dimensiones, podemos reducir a 49 componentes principales sin degradar considerablemente el dataset.

In [ ]:
np.cumsum(pca.explained_variance_ratio_)[49]

Como podemos ver, en vez de usar todas las columnas, podemos usar directamente considerablemente menos teniendo una mínima degradación del dataset.

In [ ]:
pca = PCA(n_components=49)
dataset = pca.fit_transform(data_rescaled)

Además con el fin de comparar resultados finales, aplicaremos PCA para 3 componentes.

In [ ]:
pca_deg = PCA(n_components=3)
dataset_deg = pca_deg.fit_transform(data_rescaled)

In [ ]:
dataset.shape

In [ ]:
X = dataset
X_deg = dataset_deg
#y = np.array(df_completo['anime_id'])
X.shape


In [ ]:
X_deg.shape

Teniendo nuestro dataset reescalado y reducido, podemos comenzar con el algoritmo de clustering **K-Means**

## Determinemos el número de clústers mediante el método *Elbow Curve*

In [ ]:
def elbow_curve(X):
    n_clusters = range(1, 10)
    kmeans = [KMeans(n_clusters=i) for i in n_clusters]
    kmeans
    score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
    score
    plt.plot(n_clusters,score)
    plt.xlabel('Número de clústers')
    plt.ylabel('Score')
    plt.title('Elbow Curve')
    plt.show()

Para el dataset resultante de PCA con bajo degradamiento.

In [ ]:
elbow_curve(X)

Para el dataset resultante de PCA con 3 componentes.

In [ ]:
elbow_curve(X_deg)

Los resultados mediante *Elbow Curve* no son muy claros, por lo que se utilizará el método *Silhouette Score*

## Determinemos el número de clústers mediante el método *Silhouette Score*

In [ ]:
def silhouette_score_sk(X):
    # EJEMPLO EXTRAÍDO DE SKLEARN POR EL TIPO DE GRÁFICO, créditos a sklearn.
    range_n_clusters = [2, 3, 4, 5, 6]
    for n_clusters in range_n_clusters:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')

        # Labeling the clusters
        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
        ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                    c="white", alpha=1, s=200, edgecolor='k')

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                        s=50, edgecolor='k')

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d" % n_clusters),
                     fontsize=14, fontweight='bold')

    plt.show()

Obtenemos los gráficos con ambos dataset con el fin de comparar.

In [ ]:
silhouette_score_sk(X)

In [ ]:
silhouette_score_sk(X_deg)

- Para ambos casos 2 y 3 clústers cumplen la tarea de clustering, aunque no de forma satifactoría dado que a simple vista no están bien definidos los clústers.

Para ejecutar el algoritmo de clustering se hará lo mismo, utilizar ambos datasets para comparar los grupos finales.

In [ ]:
kmeans = KMeans(n_clusters=3).fit(X)
kmeans_3comp = KMeans(n_clusters=3).fit(X_deg)

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.cluster_centers_.shape

In [ ]:
pd.Series(kmeans.predict(X)).value_counts()

In [ ]:
pd.Series(kmeans_3comp.predict(X_deg)).value_counts()

In [ ]:
pd.Series(kmeans.predict(X))

In [ ]:
pd.Series(kmeans_3comp.predict(X_deg))

In [ ]:
df_rating_animes.shape

In [ ]:
df_rating_animes['cluster_n_comp'] = kmeans.predict(X)
df_rating_animes['cluster_3_comp'] = kmeans_3comp.predict(X_deg)

In [ ]:
df_rating_animes.groupby(['cluster_n_comp']).mean()

In [ ]:
df_rating_animes.groupby(['cluster_3_comp']).mean()

A simple vista se ven cambios muy bruscos al reducir tan drásticamente el número de componentes

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_n_comp == 0].mean().sort_values(ascending=False)[:10].to_string())

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_3_comp == 0].mean().sort_values(ascending=False)[:10].to_string())

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_n_comp == 1].mean().sort_values(ascending=False)[:10].to_string())

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_3_comp == 1].mean().sort_values(ascending=False)[:10].to_string())

El clúster número 1 queda muy parecido entre ambos resultados, incluso en el mismo orden

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_n_comp == 2].mean().sort_values(ascending=False)[:10].to_string())

In [ ]:
print(df_rating_animes.loc[df_rating_animes.cluster_3_comp == 2].mean().sort_values(ascending=False)[:10].to_string())

El clúster 3 si bien tiene coincidencias entre ambos, los resultados son muy distintos.

# Conclusión

Los animes correspondientes a cada clúster son los animes que se le pueden recomendar al usuario perteneciente al clúster. Además con estos clústers obtenidos se pueden hacer más estudios con respecto a los datos presentes en cada clúster al relacionarlo con la tabla(o dataframe) *df_anime* tales como:
   - Ideintificar tipos de anime.
   - Identificar géneros.
   - Hacer un extenso análisis de los episodios en los animes de los diferentes clústers.
   - Clasificar otros usuarios.

# Falencias del modelo

Creo que el hecho de utilizar como valor 0 el no haber visto un anime en la misma variable en donde se indica el gusto(rating) por el anime es incorrecto, puesto que al indicar 0 estamos indicando que el gusto por el anime es pésimo, esto aunque sea en escala del 1 al 10, ya que el modelo no tiene conocimiento de esta escala que es una convención humana, por lo que creo que este valor 0 distorciona este modelo basado en distancias.